# Closed-loop : reading and stimulating

This tutorial details how to use the Intan to read the number of spikes in a way that may be used to perform closed-loop experiments.

The concept is that if you need to trigger stimulations based on some response of the organoids, you may use these functionalities to do so.

```{note}
If you need the activity over periods longer than minutes, you may wish to use the database instead. See the [Database tutorial](db_usage).
```

The counting will obtain the count for 200ms after the trigger is sent. The count is stored in a buffer for all 128 electrodes.

## Using the Intan to read the number of spikes

The `IntanSofware` class provides the following methods :

- `set_count` : set which triggers start the counting process
  - The input should be a list of trigger IDs (from 0 to 15).
- Next, you must send the trigger in question to start the counting process.
- Finally, `read_count` lets you retrieve the buffer storing the number of spikes.
  - The buffer is a 128-length array containing the number of spikes for each electrode. 
  - Make sure to select the channels relevant to your experiment.

```{caution}
`read_count` intrinsically does nothing to count the spikes.<br>
It only retrieves the buffer storing the number of spikes.<br>
You must first send the triggers (using a `Trigger()` object, see [here](core:np-trigger-gen)) that have been set to listen (using `set_count`) to populate the `read_count` buffer.
```

**If you set a trigger to read on which there is a StimParam defined, the StimParam will be triggered as well.**


If you only wish to read, make sure to use a trigger without a StimParam.

## Example

In [ ]:
from neuroplatform import Experiment, StimParam, IntanSofware, Trigger
from datetime import datetime, UTC
import time
import numpy as np

In [ ]:
token = "E9JTQDS62Y"  # We provide you a token for the experiment
exp = Experiment(token)
print(f"Electrodes: {exp.electrodes}")  # Electrodes that you can used

In [ ]:
stim_param1 = StimParam()
stim_param1.index = exp.electrodes[0]  # the first electrode
stim_param1.trigger_key = 0  # the first trigger
stim_param1.phase_amplitude1 = 2.0
stim_param1.phase_amplitude2 = 2.0
stim_param1.display_attributes()

In [ ]:
intan = IntanSofware()
trigger_gen = Trigger()
intan.set_count(
    [0, 1]
)  # Set the Intan to record the number of spike whenever sending trigger 0 or 1

stim_params = [stim_param1]

try:
    if exp.start():  # Signal the start of an experiment to all users
        # Send stim parameter
        intan.send_stimparam(stim_params)
        start_exp = datetime.now(UTC)

        # Create the array of triggers, and set the desired trigger to 1
        trigger0 = np.zeros(16, dtype=np.uint8)
        trigger0[0] = 1

        for i in range(10):
            # Stimulation on electrode 0
            trigger_gen.send(trigger0)  # THIS TRIGGERS BOTH THE STIM AND THE READ
            # Wait 200ms to read for a close loop
            time.sleep(0.2)
            count_spikes = intan.read_count()  # np.array(128)
            # count the number of spike of electrode 22
            total_spikes_stim = count_spikes[22]

            ###

            # Send trigger 1 - no stimulation
            trigger1 = np.zeros(16, dtype=np.uint8)
            trigger1[1] = 1
            trigger_gen.send(trigger1)  # THIS TRIGGERS ONLY THE READ
            # Wait 200ms to read for a close loop
            time.sleep(0.2)
            count_spikes = intan.read_count()  # np.array(128)
            # count the number of spike of electrode 22
            total_spikes_read_only = count_spikes[22]

            print(f"Spikes with stimulation: {total_spikes_stim}")
            print(f"Spikes without stimulation: {total_spikes_read_only}")

            time.sleep(1)

        stop_exp = datetime.utcnow()

        # Disable all parameters
        for stim in stim_params:
            stim.enable = False
        intan.send_stimparam(stim_params)

finally:
    # Close the connection to the trigger generator
    trigger_gen.close()
    # Close the connection to the Intan
    intan.close()
    # Signal the end of an experiment to all users
    exp.stop()